# Direct

In [13]:


from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import GPy

from Booster_single_obj_new import Booster

pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]

k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01


mean_cons = np.zeros((9))
std_cons = np.zeros((9))
for i in range(9):
    mean_cons[i] = np.mean(Cons[:,i])
    std_cons[i] = np.std(Cons[:, i])
ratio = mean_cons/std_cons
cdf = norm.cdf(ratio)
term = np.prod(cdf)

# perform the optimization process
# select the next point to sample
nbiter = 10
for i in range(nbiter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    mu, std = m.predict(Xsamples, full_cov=False)
    scores = ((mu - best)*norm.cdf((mu - best - xi) / (std))+std*norm.pdf((mu - best) / (std)))*term
    ix = np.argmax(scores)
    x = Xsamples[ix, :][:,None].T
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]
    
    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])
    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
    m = GPy.models.GPRegression(X, y, k)

ix = np.argmax(y)
print('optimum: x1=%.3f, x2=%.3f, x3=%.3f, x4=%.3f, y=%.3f' % (X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))

optimum: x1=0.799, x2=0.898, x3=0.115, x4=0.967, y=5.281


# EI

In [14]:


pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]

#y = np.array([objective(x1,x2) for (x1,x2) in zip(X[:,0], X[:,1])])[:,None]
k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01


# perform the optimization process
# select the next point to sample
iter = 60
for i in range(iter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    # calculate the acquisition function for each sample

    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = m.predict(Xsamples, full_cov=False)
    # calculate the probability of improvement
    scores = (mu - best)*norm.cdf((mu - best - xi) / (std))+std*norm.pdf((mu - best) / (std))
    # locate the index of the largest scores
    ix = np.argmax(scores)
    x = Xsamples[ix, :][:,None].T
    # sample the point
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]
    
    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])
   
        # summarize the finding
    #est, _ = m.predict([[x]], full_cov=False)
    #print("est: ",est)
    #print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset
    
    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
        # update the model
    m = GPy.models.GPRegression(X, y, k)

# best result
ix = np.argmax(y)
print('Best Result: x1=%.3f, x2=%.3f, x3=%.3f, x4=%.3f, y=%.3f' % (X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))



Best Result: x1=0.024, x2=0.136, x3=0.280, x4=0.883, y=5.186


# UCB

In [15]:
pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]

k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01

# perform the optimization process
# select the next point to sample
iters = 60

for i in range(iter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    # calculate the acquisition function for each sample

    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = m.predict(Xsamples, full_cov=False)
    # calculate the probability of improvement
    scores = mu + xi*std
    # locate the index of the largest scores
    ix = np.argmax(scores)
    x = Xsamples[ix, :][:,None].T
    # sample the point
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]

    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])

        # summarize the finding
    #est, _ = m.predict([[x]], full_cov=False)
    #print("est: ",est)
    #print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset

    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
        # update the model
    m = GPy.models.GPRegression(X, y, k)

# best result
ix = np.argmax(y)
print('Best Result: x1=%.3f, x2=%.3f, x3=%.3f, x4=%.3f, y=%.3f' % (X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))





Best Result: x1=0.087, x2=0.604, x3=0.308, x4=0.835, y=4.923


# POF

In [16]:
pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]


#y = np.array([objective(x1,x2) for (x1,x2) in zip(X[:,0], X[:,1])])[:,None]
k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01

mean_cons = np.zeros((9))
std_cons = np.zeros((9))
for i in range(9):
    mean_cons[i] = np.mean(Cons[:,i])
    std_cons[i] = np.std(Cons[:, i])
ratio = mean_cons/std_cons
cdf = norm.cdf(ratio)
term = np.prod(cdf)

# perform the optimization process
# select the next point to sample
iter = 60
for i in range(iter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    # calculate the acquisition function for each sample

    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = m.predict(Xsamples, full_cov=False)
    # calculate the probability of improvement
    scores = (mu - best)*norm.cdf((mu - best - xi) / (std))+std*norm.pdf((mu - best) / (std)) * term
    # locate the index of the largest scores
    ix = np.argmax(scores)
    x = Xsamples[ix, :][:,None].T
    # sample the point
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]
    
    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])
   
        # summarize the finding
    #est, _ = m.predict([[x]], full_cov=False)
    #print("est: ",est)
    #print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset
    
    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
        # update the model
    m = GPy.models.GPRegression(X, y, k)

# best result
ix = np.argmax(y)
print('Best Result: x1=%.3f, x2=%.3f, x3=%.3f, x4=%.3f, y=%.3f' % (X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))



Best Result: x1=0.237, x2=0.294, x3=0.196, x4=0.731, y=5.073


# PI

In [17]:
pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]


#y = np.array([objective(x1,x2) for (x1,x2) in zip(X[:,0], X[:,1])])[:,None]
k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01


# perform the optimization process
# select the next point to sample
iter = 60
for i in range(iter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    # calculate the acquisition function for each sample

    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = m.predict(Xsamples, full_cov=False)
    # calculate the probability of improvement
    scores = norm.cdf((mu - best - xi) / (std+1E-9))
    # locate the index of the largest scores
    ix = np.argmax(scores)
    x = Xsamples[ix, :][:,None].T
    # sample the point
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]
    
    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])
   
        # summarize the finding
    #est, _ = m.predict([[x]], full_cov=False)
    #print("est: ",est)
    #print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset
    
    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
        # update the model
    m = GPy.models.GPRegression(X, y, k)

# best result
ix = np.argmax(y)
print('Best Result: x1=%.3f, x2=%.3f, x3=%.3f, x4=%.3f, y=%.3f' % (X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))



Best Result: x1=0.391, x2=0.472, x3=0.258, x4=0.828, y=5.007
